<a href="https://colab.research.google.com/github/pablo-arantes/Cloud-Bind/blob/main/GNINA%2BOpenBPMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hi there!**

This is a Jupyter notebook for running molecular docking calculations with deep learning using Gnina docking software, which utilizes an ensemble of convolutional neural networks as a scoring function, and for running OpenBPMD, to evaluate the ligand pose stability using metadynamics. OpenBPMD is powered by the OpenMM simulation engine and uses a revised scoring function.

The main goal of this notebook is to demonstrate how to harness the power of cloud-computing to perform drug binding structure prediction in a cheap and yet feasible fashion.

---

 **This notebook is NOT a standard protocol for docking calculations and MD simulations!** It is just simple docking pipeline illustrating each step of a docking and MD protocol.

--- 


**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/pablo-arantes/Cloud-Bind/issues

**Acknowledgments**
- We would like to thank the [GNINA](https://github.com/gnina/gnina) team for doing an excellent job open sourcing the software.
- We would like to thank the [Roitberg](https://roitberg.chem.ufl.edu/) team for developing the fantastic [TorchANI](https://github.com/aiqm/torchani).
- We would like to thank [@ruiz_moreno_aj](https://twitter.com/ruiz_moreno_aj) for his work on [Jupyter Dock](https://github.com/AngelRuizMoreno/Jupyter_Dock) 
- We would like to thank the ChemosimLab ([@ChemosimLab](https://twitter.com/ChemosimLab)) team for their incredible [ProLIF](https://prolif.readthedocs.io/en/latest/index.html#) (Protein-Ligand Interaction Fingerprints) tool.
- We would like to thank the [OpenBPMD](https://github.com/Gervasiolab/OpenBPMD) team for their open source implementation of binding pose metadynamics (BPMD).
- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.
- Finally, we would like to thank [Making it rain](https://github.com/pablo-arantes/making-it-rain) team, **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)), for their amazing work.
- A Cloud-Bind by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes))

- For related notebooks see: [Cloud-Bind](https://github.com/pablo-arantes/Cloud-Bind)

In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
# install dependencies
%%capture
import sys
import tarfile
import os 

!pip -q install py3Dmol
!pip install git+https://github.com/pablo-arantes/biopandas
!pip install bio
!mamba install openmmforcefields -c conda-forge -y
!pip install prolif
!mamba install pytorch torchvision -c pytorch
!pip install torchani
!pip install ase
!mamba install -c conda-forge pdbfixer -y
!mamba install -c conda-forge parmed -y
!mamba install -c conda-forge openbabel -y
!pip install --upgrade MDAnalysis
!wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina
!chmod +x gnina
!wget https://github.com/rdk/p2rank/releases/download/2.4/p2rank_2.4.tar.gz
file = tarfile.open('p2rank_2.4.tar.gz')
file.extractall('/content/')  
file.close()
os.remove('p2rank_2.4.tar.gz')

#load dependencies
from openmm import app, unit
from openmm.app import HBonds, NoCutoff, PDBFile
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path
import parmed as pmd
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import urllib.request  
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats

## Using Google Drive to store simulation data

Google Colab does not allow users to keep data on their computing nodes. However, we can use Google Drive to read, write, and store our simulations files. Therefore, we suggest to you to:

1.   Create a folder in your own Google Drive and copy the necessary input files there.
2.   Copy the path of your created directory. We will use it below.

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Check if you correctly allocated GPU nodes**

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


---
# **Loading the necessary input files**

At this point, we should have all libraries and dependencies installed.

**Important**: Make sure the PDB file points to the correct structure.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

**Please, don't use spaces in the files and folders names, i.e. MyDrive/protein_ligand and so on.**

In [ ]:
#@title **Please, provide the necessary input files below for receptor**: 
#@markdown **Important:** Run the cell to prepare your receptor and select your reference residue for the construction of an optimal box size for the docking calculations.
from openmm.app.pdbfile import PDBFile

import warnings
warnings.filterwarnings('ignore')
import os
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB import is_aa
import pandas as pd
from pdbfixer import PDBFixer

Google_Drive_Path = '/content/' #@param {type:"string"}
workDir = Google_Drive_Path

workDir2 = os.path.join(workDir)
workDir_check = os.path.exists(workDir2)
if workDir_check == False:
  os.mkdir(workDir2)
else:
  pass

if os.path.exists(os.path.join(workDir, "name_residues.txt")):
  os.remove(os.path.join(workDir, "name_residues.txt"))
  os.remove(os.path.join(workDir,"name_residues_receptor.txt"))
else:
  pass

temp = os.path.join(workDir, "temp.pdb")
receptor = os.path.join(workDir, "receptor.pdb")
ligand = os.path.join(workDir, "ligand.sdf")


Query_PDB_ID = '3HTB' #@param {type:"string"}

pdbfn = Query_PDB_ID + ".pdb"
url = 'https://files.rcsb.org/download/' + pdbfn
outfnm = os.path.join(workDir, pdbfn)
urllib.request.urlretrieve(url, outfnm)


ppdb = PandasPdb().read_pdb(outfnm)
ppdb.df['ATOM'] = ppdb.df['ATOM']
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']
ppdb.to_pdb(path=temp, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

#prepare receptor
ppdb = PandasPdb().read_pdb(outfnm)
ppdb.df['ATOM'] = ppdb.df['ATOM']
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.df['ATOM']= ppdb.df['ATOM'][ppdb.df['ATOM']['element_symbol'] != 'H']
ppdb.to_pdb(path=receptor, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

fixer = PDBFixer(filename=receptor)
fixer.removeHeterogens()
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.4)
PDBFile.writeFile(fixer.topology, fixer.positions, open(receptor, 'w'))


path = '/content/'


def is_het(residue):
    res = residue.id[0]
    return res != " " and res != "W"

def aa(residue):
    res = residue.id[0]
    return res != "W" 


class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        return residue == self.residue and aa(residue)

def extract_ligands(path):
    pdb = PDBParser().get_structure(temp, temp)
    io = PDBIO()
    io.set_structure(pdb)
    i = 1
    name_residues = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues.append(residue)
          print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues.txt"), "a",))
          i += 1

extract_ligands(path)

def extract_ligands2(path):
    pdb = PDBParser().get_structure(receptor, receptor)
    io = PDBIO()
    io.set_structure(pdb)
    i2 = 1
    name_residues2 = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues2.append(residue)
          print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues_receptor.txt"), "a",))
          i2 += 1

extract_ligands2(path)


dataset = pd.read_csv(os.path.join(workDir, 'name_residues.txt'), delimiter = " ", header=None)
df = pd.DataFrame(dataset)
df = df.iloc[:, [2]]
new = df.to_numpy()

dataset2 = pd.read_csv(os.path.join(workDir, 'name_residues_receptor.txt'), delimiter = " ", header=None)
df2 = pd.DataFrame(dataset2)
df2 = df2.iloc[:, [2]]
new2 = df2.to_numpy()

b = 1
res_number = []
for j in new2:
  res_number.append(b)
  b += 1

print("Residue" + " - "  + "Number" )
a = 1
for j in new:
  print(', '.join(j) + " - "  + str(a))
  a += 1

  # print(number_residues)

In [ ]:
#@title **Predict ligand-binding pockets from your protein structure using P2Rank**: 
#@markdown **P2Rank** is a stand-alone command line program that predicts ligand-binding pockets from a protein structure. It achieves high prediction success rates without relying on an external software for computation of complex features or on a database of known protein-ligand templates.
#@markdown P2Rank makes predictions by scoring and clustering points on the protein's solvent accessible surface. Ligandability score of individual points is determined by a machine learning based model trained on the dataset of known protein-ligand complexes. For more details see [here](https://github.com/rdk/p2rank).

import subprocess
import csv

output_p2rank = os.path.join(workDir, "output_p2rank")
p2rank = "/content/p2rank_2.4/prank predict -f " + str(receptor) + " -o " + str(output_p2rank)
original_stdout = sys.stdout
with open('p2rank.sh', 'w') as f:
  sys.stdout = f
  print(p2rank)
  sys.stdout = original_stdout
subprocess.run(["chmod 700 p2rank.sh"], shell=True)
subprocess.run(["./p2rank.sh"], shell=True,)

with open(os.path.join(workDir, "output_p2rank/receptor.pdb_predictions.csv"), 'r') as file:
  csvreader = csv.reader(file)
  residue = []
  score = []
  center_x = []
  center_y = []
  center_z = []
  for row in csvreader:
    residue.append(row[9:10])
    score.append(row[2:3])
    center_x.append(row[6:7])
    center_y.append(row[7:8])
    center_z.append(row[8:9])

for i in range(1,len(residue)):
  file = str((residue[i])[0]).split()
  score_end = str((score[i])[0]).split()
  center_x_end = str((center_x[i])[0]).split()
  center_y_end = str((center_y[i])[0]).split()
  center_z_end = str((center_z[i])[0]).split()
  print("Pocket " + str(i))
  print("Score = " + score_end[0])
  final_residues = []
  for i in range(0,len(file)):
    test = file[i]
    final_residues.append(int(test[2:]))
  print("Selected Residues = " + str(final_residues))
  print("Center x = "+ str(center_x_end[0]), "Center y = "+ str(center_y_end[0]), "Center z = "+ str(center_z_end[0]) + "\n")

In [ ]:
#@title **Please, provide the pocket or residue number for the selection**: 
#@markdown **Important:** The selected pocket or residues will be used as a reference for the construction of an optimal box size for the ligand during the docking. If you want to select more than one residue, please, use comma to separte the numbers (i.e. 147,150,155,160). **Please, DO NOT USE SPACES BETWEEN THEM.**

import re
import csv

if os.path.exists(os.path.join(workDir, "name_residue.txt")):
  os.remove(os.path.join(workDir, "name_residue.txt"))
else:
  pass
  
# Python code to convert string to list
def Convert(string):
	li = list(string.split(","))
	return li

def extract_ligands(path,residues):
    pdb = PDBParser().get_structure(temp, temp)
    io = PDBIO()
    io.set_structure(pdb)
    i = 1
    name_residues = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          if i == int(residues):
            # print(residues)
            print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residue.txt"), "a",))
            io.save(f"res_{i}_certo.pdb", ResidueSelect(chain, residue))
          i += 1

Selection = "Pocket" #@param ["Pocket", "Residues"]

number = '2' #@param {type:"string"}

if Selection == "Pocket":
  file = str((residue[int(number)])[0]).split()
  score_end = str((score[int(number)])[0]).split()
  center_x_end = str((center_x[int(number)])[0]).split()
  center_y_end = str((center_y[int(number)])[0]).split()
  center_z_end = str((center_z[int(number)])[0]).split()
  center_x_gnina = float(center_x_end[0])
  center_y_gnina = float(center_y_end[0])
  center_z_gnina = float(center_z_end[0])
  print("Pocket " + str(number))
  print("Score = " + score_end[0])
  print("Center x = "+ str(center_x_end[0]), "Center y = "+ str(center_y_end[0]), "Center z = "+ str(center_z_end[0]) + "\n")
  final_residues = []
  for i in range(0,len(file)):
    test = file[i]
    final_residues.append(int(test[2:]))
  residues_num = final_residues
else:
  residues_num = Convert(number)

filenames=[]
for k in range(0, len(residues_num)):  
  extract_ligands(path, residues_num[k])
  filenames.append(f"res_{residues_num[k]}_certo.pdb")


with open('selection_merge.pdb', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

# reading each line from original
# text file
file1 = open('/content/selection_merge.pdb', 'r')
file2 = open('/content/selection_merge_end.pdb','w')

for line in file1.readlines():
   
    # reading all lines that begin
    # with "TextGenerator"
    x = re.findall("^END", line)
     
    if not x:
        file2.write(line)
         
# close and save the files
file1.close()
file2.close()

dataset = pd.read_csv(os.path.join(workDir, "name_residue.txt"), delimiter = " ", header=None)
df = pd.DataFrame(dataset)
df = df.iloc[:, [2]]
new = df.to_numpy()

print("Selected Residue" + " - "  + "Number" )
for j, i in zip(new, range(0, len(residues_num))):
# for j in new:
  print(', '.join(j) + " - "  + str(residues_num[i]))
res_box = '/content/selection_merge_end.pdb'

In [ ]:
#@title **Receptor Visualization**: 
#@markdown Now the protein has been sanitized and the selection has been chosen, it would be recomended to visualize and check the protein (gray) and your selection (green).

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(receptor,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open(res_box,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
#@title **Please, provide the necessary input files for the ligand**: 

#@markdown Type the smiles or filename (PDB and MOL formats) of your molecule. **Ex: C=CC(=O)OC, molecule.pdb or molecule.mol** 

#@markdown Just remind you that if you want to use pdb or mol file, you should first upload the file here in Colab or in your Google Drive.

#@markdown If you don't know the exactly smiles, please, check at https://pubchem.ncbi.nlm.nih.gov/

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolTransforms
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor
from IPython.display import SVG
import ipywidgets as widgets
import rdkit
from rdkit.Chem.Draw import IPythonConsole
AllChem.SetPreferCoordGen(True)
from IPython.display import Image
from openbabel import pybel

import os

import py3Dmol


Type = "smiles" #@param ["smiles", "pdb", "mol"]

smiles_or_filename = 'CCCC1=CC=CC=C1O  ' #@param {type:"string"}
workDir = "/content/"

if Type == "smiles":
  Smiles = smiles_or_filename 
  smiles_fig = Chem.MolFromSmiles(Smiles)
  hmol = Chem.AddHs(smiles_fig)
  AllChem.EmbedMolecule(hmol)
  hmol.GetConformer(0)
  mp = AllChem.MMFFGetMoleculeProperties(hmol)
  ff = AllChem.MMFFGetMoleculeForceField(hmol, mp)
  # Optimize
  AllChem.OptimizeMoleculeConfs(hmol, ff, numThreads=1, maxIters=1000)
  AllChem.MolToMolFile(hmol, (os.path.join(workDir, "ligand.mol")))
  AllChem.MolToPDBFile(hmol, (os.path.join(workDir, "ligand.pdb")))
elif Type == "pdb":
  mol= [m for m in pybel.readfile(filename=smiles_or_filename, format='pdb')][0]
  out=pybel.Outputfile(filename='mol.mol',format='mol',overwrite=True)
  out.write(mol)
  out.close()
  mol = Chem.MolFromMolFile('mol.mol')
  Smiles = Chem.MolToSmiles(mol)
  smiles_fig = Chem.MolFromSmiles(Smiles)
  hmol = Chem.AddHs(smiles_fig)
  AllChem.EmbedMolecule(hmol)
  hmol.GetConformer(0)
  mp = AllChem.MMFFGetMoleculeProperties(hmol)
  ff = AllChem.MMFFGetMoleculeForceField(hmol, mp)
  # Optimize
  AllChem.OptimizeMoleculeConfs(hmol, ff, numThreads=1, maxIters=1000)
  AllChem.MolToMolFile(hmol, (os.path.join(workDir, "ligand.mol")))
  AllChem.MolToPDBFile(hmol, (os.path.join(workDir, "ligand.pdb")))
else:
  mol = Chem.MolFromMolFile(smiles_or_filename)
  Smiles = Chem.MolToSmiles(mol)
  smiles_fig = Chem.MolFromSmiles(Smiles)
  hmol = Chem.AddHs(smiles_fig)
  AllChem.EmbedMolecule(hmol)
  hmol.GetConformer(0)
  mp = AllChem.MMFFGetMoleculeProperties(hmol)
  ff = AllChem.MMFFGetMoleculeForceField(hmol, mp)
  # Optimize
  AllChem.OptimizeMoleculeConfs(hmol, ff, numThreads=1, maxIters=1000)
  AllChem.MolToMolFile(hmol, (os.path.join(workDir, "ligand.mol")))
  AllChem.MolToPDBFile(hmol, (os.path.join(workDir, "ligand.pdb")))

print("Smiles: " + str(Smiles))
smi = Draw.MolToFile(smiles_fig, size=(600, 600), filename=os.path.join(workDir,str(Smiles) + '.png'))
img = mpimg.imread(os.path.join(workDir,str(Smiles) + '.png'))
plt.figure(figsize = (8,8))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
from typing import List
from ase import Atoms
from ase.lattice.cubic import Diamond
from ase.md.langevin import Langevin
from ase.optimize import BFGS
from ase import io
from ase.io import read, write
from ase import units
from ase.constraints import ExternalForce, FixInternals
import torch
import torchani
import pandas as pd
import numpy as np
from openbabel import pybel
from torchani.units import HARTREE_TO_KCALMOL


#@title **Ligand geometry optimization using TorchANI**: 

#@markdown Geometry optimization for the ligand 3D structure, using ANI-1x, ANI-1ccx or ANI-2x as the optimizing engine.

#@markdown If you want to know more about **TorchANI**, please, check at https://aiqm.github.io/torchani/

model_name = "ANI-2x" #@param ["ANI-1x", "ANI-1ccx", "ANI-2x"]

#@markdown Convergence threshold for geometry optimization:

opt_tol = 0.0001 #@param {type:"slider", min:0.0001, max:0.01, step:0.0001}



# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

if model_name == "ANI-2x":
  model = torchani.models.ANI2x(periodic_table_index=True).to(device)
  calculator = torchani.models.ANI2x().ase()
  print("Model = ANI2x")
elif model_name == "ANI-1ccx":
  model = torchani.models.ANI1ccx(periodic_table_index=True).to(device)
  calculator = torchani.models.ANI1ccx().ase()
  print("Model = ANI1ccx")
elif model_name == "ANI-1x":
  model = torchani.models.ANI1x(periodic_table_index=True).to(device)
  calculator = torchani.models.ANI1x().ase()
  print("Model = ANI1x")
else:
  pass

def mol2arr(mols, device=device):
    coordinates = []
    spices = []
    for mol in mols:
        pos = mol.GetConformer().GetPositions().tolist()
        atomnums = [a.GetAtomicNum() for a in mol.GetAtoms()]
        coordinates.append(pos)
        spices.append(atomnums)
    coordinates = torch.tensor(coordinates,
                               requires_grad=True,
                               device=device)
    species = torch.tensor(spices, device=device)
    return coordinates, species

mol_deg = AllChem.MolFromMolFile ((os.path.join(workDir, "ligand.mol")), removeHs=False)
mol = io.read(os.path.join(workDir, "ligand.mol"))
coordinates, species = mol2arr([mol_deg], device)
tensor1 = coordinates.detach().numpy()
atoms = Atoms(mol, positions=tensor1[0])
atoms.center(vacuum=3.0)
atoms.set_calculator(calculator)
print("Begin Geometry Optimization ")
opt = BFGS(atoms)
opt.run(fmax=opt_tol)
# print()
write((os.path.join(workDir, "ligand_min.xyz")), format="xyz", images=atoms)

atomic_symbols = []
xyz_coordinates = []

with open((os.path.join(workDir, "ligand_min.xyz")), "r") as file:
  for line_number,line in enumerate(file):
      if line_number == 0:
          num_atoms = int(line)
      elif line_number == 1:
          comment = line # might have useful information
      else:
          atomic_symbol, x, y, z = line.split()
          atomic_symbols.append(atomic_symbol)
          xyz_coordinates.append([float(x),float(y),float(z)])

from rdkit.Geometry import Point3D
conf = mol_deg.GetConformer()

for i in range(mol_deg.GetNumAtoms()):
  x,y,z = xyz_coordinates[i]
  conf.SetAtomPosition(i,Point3D(x,y,z))
AllChem.MolToMolFile(mol_deg, (os.path.join(workDir, "ligand_min.mol")))
AllChem.MolToPDBFile(mol_deg, (os.path.join(workDir, "ligand_min.pdb")))
#convert to sdf format
mol= [m for m in pybel.readfile(filename=os.path.join(workDir, "ligand_min.mol"), format='mol')][0]
out=pybel.Outputfile(filename=ligand,format='sdf',overwrite=True)
out.write(mol)
out.close()


#TorchANI Energies
mol_deg = AllChem.MolFromMolFile ((os.path.join(workDir, "ligand_min.mol")), removeHs=False)
coordinates, species = mol2arr([mol_deg], device)
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative
print('Energy:', energy.item())
# print('Force:', force.squeeze())

In [ ]:
#@title **Ligand Visualization**: 
#@markdown Now the ligand has been optimized, it would be recomended to visualize and check the ligand.

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.05})

# view.addModel(open(receptor,'r').read(),format='pdb')
# Prot=view.getModel()
# Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
# view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open(ligand,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
#@title **Parameters for the docking calculation:**
Output_name = 'output_docking' #@param {type:"string"}
#@markdown Amount of buffer space to add the generated box (Angstroms):
size = 4 #@param {type:"slider", min:1, max:20, step:1}

#@markdown Exhaustiveness of the global search (roughly proportional to time):
exhaustiveness = 8 #@param {type:"slider", min:2, max:64, step:2}

#@markdown Explicit random seed:
seed = "0" #@param {type:"string"}

#@markdown Convolutional neural network (CNN) parameter:

cnn_scoring = "rescore (default)" #@param ["none", "rescore (default)", "refinement", "all"]
if cnn_scoring == "rescore (default)":
  cnn_scoring = "rescore"
  scoring_vinardo = " "
elif cnn_scoring == "none":
  scoring_vinardo = " --scoring vinardo "
else:
  scoring_vinardo = " "

# if cnn_scoring == "none":
#   scoring_vinardo = " --scoring vinardo "
# else:
#   scoring_vinardo = " "

#@markdown **cnn_scoring** determines at what points of the docking procedure that the CNN scoring function is used.

#@markdown **none** - No CNNs used for docking. Here, uses all the empirical scoring from [Vinardo](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0155183) scoring function.

#@markdown **rescore** (default) - CNN used for reranking of final poses. Least computationally expensive CNN option.

#@markdown **refinement** - CNN used to refine poses after Monte Carlo chains and for final ranking of output poses. 10x slower than rescore when using a GPU.

#@markdown **all** - CNN used as the scoring function throughout the whole procedure. Extremely computationally intensive and not recommended.

#@markdown The default CNN scoring function is an ensemble of 5 models selected to balance pose prediction performance and runtime: dense, general_default2018_3, dense_3, crossdock_default2018, and redock_default2018. 

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

docking_output_gz = os.path.join(workDir, Output_name + ".sdf.gz")
docking_output = os.path.join(workDir, Output_name + ".sdf")

if os.path.exists(docking_output_gz):
  os.remove(docking_output_gz)
elif os.path.exists(docking_output):
  os.remove(docking_output)
else:
  pass


if Selection == "Pocket":
  gnina = "./gnina -r " + str(receptor) + " -l " +  str(ligand) + " --center_x " + str(center_x_gnina) +  " --center_y " + str(center_y_gnina) +  " --center_z " + str(center_z_gnina) + " --size_x " + str(size) +  " --size_y " + str(size) +  " --size_z " + str(size) + " --cnn_scoring " + str(cnn_scoring) + " --exhaustiveness " + str(exhaustiveness) + " -o " + str(docking_output_gz) + str(scoring_vinardo) +  "--num_modes 10 " + "--seed " + str(int(seed))
else:
  gnina = "./gnina -r " + str(receptor) + " -l " +  str(ligand) + " --autobox_ligand " + str(res_box) +  " --autobox_add " + str(size) + " --cnn_scoring " + str(cnn_scoring) + " --exhaustiveness " + str(exhaustiveness) + " -o " + str(docking_output_gz) + str(scoring_vinardo) +  "--num_modes 10 " + "--seed " + str(int(seed))

zip_gnina = "gunzip " + str(docking_output_gz)

original_stdout = sys.stdout # Save a reference to the original standard output
with open('gnina.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(gnina)
    print(zip_gnina)
    sys.stdout = original_stdout # Reset the standard output to its original value

!chmod 700 gnina.sh 2>&1 1>/dev/null
!bash gnina.sh

import gzip
v = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
v.setViewStyle({'style':'outline','color':'black','width':0.05})
v.addModel(open(receptor).read())
v.setStyle({'cartoon':{},'stick':{'colorscheme':'white','radius':.1}})
v.addModel(open(res_box).read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.175}})
v.addModelsAsFrames(open(docking_output,'rt').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)

In [ ]:
import pandas as pd
from rdkit.Chem import rdFMCS,AllChem, Draw, PandasTools
import seaborn as sns
#@title **Docking Analysis:**
Parameter = "Affinity" #@param ["Affinity", "CNN pose score", "CNN affinity"]

Output_name = 'Affinity' #@param {type:"string"}

VinaPoses=PandasTools.LoadSDF(docking_output)

AllPoses=pd.concat([VinaPoses])
# AllPoses.sort_values(by='CNNscore',inplace=False)
# AllPoses['minimizedAffinity']=[float(i) for i in AllPoses['minimizedAffinity']]
# AllPoses.sort_values(by='minimizedAffinity',inplace=True, ascending=False)
# AllPoses.reset_index(drop=True, inplace=True)
# x='CNNscore'

parameter = Parameter
if parameter == "Affinity":
  AllPoses['minimizedAffinity']=[float(i) for i in AllPoses['minimizedAffinity']]
  y='minimizedAffinity'
elif parameter == "CNN pose score":
  AllPoses['CNNscore']=[float(i) for i in AllPoses['CNNscore']]
  y='CNNscore'
else:
  AllPoses['CNNaffinity']=[float(i) for i in AllPoses['CNNaffinity']]
  y='CNNaffinity'

plt.rcParams['axes.linewidth'] = 2
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
# print(AllPoses)

a = 0
x_new =[]
for i in AllPoses['minimizedAffinity']:
  a = a + 1
  x_new.append(a)

# Keyword arguments for styling the plot
kwargs = dict (linestyle='--', color='dodgerblue', marker ='o', linewidth=0.8, markersize=15)

# Draw the plot
line = sns.lineplot(x=x_new,y=y, data=AllPoses,**kwargs)

if parameter == "Affinity":
  plt.ylabel('Affinity (Kcal/mol)',fontsize=16,fontweight='bold')
elif parameter == "CNN pose score":
  plt.ylabel('CNN pose score',fontsize=16,fontweight='bold')
else:
  plt.ylabel('CNN affinity',fontsize=16,fontweight='bold')

plt.xlabel('Mode Number',fontsize=16,fontweight='bold')
plt.xticks(x_new[::1], fontsize = 12) # set divisor
plt.yticks(fontsize = 12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()
plt.tick_params ('both',width=2,labelsize=14)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Best pose selection:**

mode_number = "1" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.05})

view.addModel(open(receptor,'r').read(),'pdb')
Prot=view.getModel()
# Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'},'stick':{'radius':.1}})
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.2,'color':'white'})


view.addModel(open(res_box,'r').read(),'pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'grayCarbon','radius':0.2}})


results=Chem.SDMolSupplier(docking_output,True)
p=Chem.MolToMolBlock(results[(int(mode_number)-1)],False)
p2=Chem.MolToMolFile(results[(int(mode_number)-1)],(os.path.join(workDir, str(int(mode_number)) + "_pose.sdf")))

print('Reference: Gray | gnina Pose: Green')
if cnn_scoring == "none":
  print ('Affinity: {}'.format(results[(int(mode_number)-1)].GetProp('minimizedAffinity')))
else:
  print ('Affinity: {}'.format(results[(int(mode_number)-1)].GetProp('minimizedAffinity')))
  print ('CNN Score: {}'.format(results[(int(mode_number)-1)].GetProp('CNNscore')))
  print ('CNN Affinity: {}'.format(results[(int(mode_number)-1)].GetProp('CNNaffinity')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
import parmed as pmd
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from openmmforcefields.generators import GAFFTemplateGenerator

# Imports from the toolkit
import openff.toolkit
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
# Imports from dependencie
from openff.units import unit
from openff.units.openmm import to_openmm, from_openmm
try:
    import openmm
    from openmm import unit
except ImportError:
    from simtk import openmm, unit

#@title **Parameters to generate the topology:**

#@markdown **Parameters to generate the protein topology:**

Force_field = "AMBER99SB" #@param ["AMBER14SB", "AMBER99", "AMBER99SB"]
Water_type = "TIP3P" #@param ["TIP3P", "SPC/E"]

#@markdown Padding distance (angstrons) determines the largest size of the solute along any axis (x, y, or z). It then creates a cubic box of width (solute size)+2*(padding). The above line guarantees that no part of the solute comes closer than 1 nm to any edge of the box.

Padding_distance = 4 #@param {type:"slider", min:1, max:10, step:1}

#@markdown **ATTENTION**: Give the concentration in Molar units:

Ions = "NaCl" #@param ["NaCl", "KCl" ]
Concentration = "0.15" #@param {type:"string"}

#@markdown **Function to add missing hydrogen atoms based on pH:**

pH = "7.4" #@param {type:"string"}

#@markdown **Parameters to generate the ligand topology:**

Ligand_Force_field = "gaff-2.11" #@param ['gaff-1.4', 'gaff-1.8', 'gaff-1.81', 'gaff-2.1', 'gaff-2.11', 'smirnoff99Frosst-1.1.0', 'smirnoff99Frosst-1.0.0', 'smirnoff99Frosst-1.0.9', 'smirnoff99Frosst-1.0.1', 'smirnoff99Frosst-1.0.6', 'smirnoff99Frosst-1.0.5', 'smirnoff99Frosst-1.0.7', 'smirnoff99Frosst-1.0.8', 'smirnoff99Frosst-1.0.2', 'smirnoff99Frosst-1.0.4', 'smirnoff99Frosst-1.0.3', 'openff-1.0.0-RC1', 'openff-1.0.1', 'openff-1.3.1-alpha.1', 'openff-1.0.0', 'openff-1.2.0', 'openff-2.0.0-rc.2', 'openff-2.0.0-rc.1', 'openff-2.0.0', 'openff-1.1.0', 'openff-1.3.1', 'openff-1.1.1', 'openff-1.0.0-RC2', 'openff-1.3.0', 'openff-1.2.1']

lig_end = (os.path.join(workDir, str(int(mode_number)) + "_ligand.sdf"))

results=Chem.SDMolSupplier(docking_output,True)
p=Chem.MolToMolFile(results[(int(mode_number)-1)],(os.path.join(workDir, str(int(mode_number)) + "_ligand.sdf")))

# Load a molecule from a SDF file
ligand = Molecule.from_file(os.path.join(workDir, str(int(mode_number)) + "_ligand.sdf"))
# Molecule stores both the co-ordinates of the atoms and their bond graph
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()

# if Force_field == "CHARMM36" and Water_type == "TIP3P":
#   water = "charmm36/water.xml"
#   water_model = "tip3p"
#   ff = "charmm36.xml"
if Force_field == "AMBER14SB" and Water_type == "TIP3P":
  water = "amber14/tip3p.xml"
  water_model = "tip3p"
  ff = "amber14/protein.ff14SB.xml"
elif Force_field == "AMBER99SB" and Water_type == "TIP3P":
  water = "tip3p.xml"
  water_model = "tip3p"
  ff = "amber99sbildn.xml"
elif Force_field == "AMBER99" and Water_type == "TIP3P":
  water = "tip3p.xml"
  water_model = "tip3p"
  ff = "amber99sb.xml"
# elif Force_field == "CHARMM36" and Water_type == "SPC/E":
#   water = "charmm36/spce.xml"
#   water_model = "spce"
#   ff = "charmm36.xml"
elif Force_field == "AMBER14SB" and Water_type == "SPC/E":
  water = "amber14/spce.xml"
  water_model = "spce"
  ff = "amber14/protein.ff14SB.xml"
elif Force_field == "AMBER99SB" and Water_type == "SPC/E":
  water = "spce.xml"
  water_model = "spce"
  ff = "amber99sbildn.xml"
elif Force_field == "AMBER99" and Water_type == "SPC/E":
  water = "spce.xml"
  water_model = "spce"
  ff = "amber99sb.xml"
else:
  pass

# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField(ff, water)

small_ff_options = ['gaff-1.4', 'gaff-1.8', 'gaff-1.81', 'gaff-2.1', 'gaff-2.11', 'smirnoff99Frosst-1.1.0', 'smirnoff99Frosst-1.0.0', 'smirnoff99Frosst-1.0.9', 'smirnoff99Frosst-1.0.1', 
            'smirnoff99Frosst-1.0.6', 'smirnoff99Frosst-1.0.5', 'smirnoff99Frosst-1.0.7', 'smirnoff99Frosst-1.0.8', 'smirnoff99Frosst-1.0.2', 'smirnoff99Frosst-1.0.4', 
            'smirnoff99Frosst-1.0.3', 'openff-1.0.0-RC1', 'openff-1.0.1', 'openff-1.3.1-alpha.1', 'openff-1.0.0', 'openff-1.2.0', 'openff-2.0.0-rc.2', 'openff-2.0.0-rc.1', 
            'openff-2.0.0', 'openff-1.1.0', 'openff-1.3.1', 'openff-1.1.1', 'openff-1.0.0-RC2', 'openff-1.3.0', 'openff-1.2.1']

type_generator_options = ('GAFF', 'GAFF', 'GAFF', 'GAFF', 'GAFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF',
                          'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF', 'SMIRNOFF')

small_ff_include = ["'gaff-1.4'", "'gaff-1.8'", "'gaff-1.81'", "'gaff-2.1'", "'gaff-2.11'", "'smirnoff99Frosst-1.1.0'", "'smirnoff99Frosst-1.0.0'", "'smirnoff99Frosst-1.0.9'", "'smirnoff99Frosst-1.0.1'", 
            "'smirnoff99Frosst-1.0.6'", "'smirnoff99Frosst-1.0.5'", "'smirnoff99Frosst-1.0.7'", "'smirnoff99Frosst-1.0.8'", "'smirnoff99Frosst-1.0.2'", "'smirnoff99Frosst-1.0.4'", 
            "'smirnoff99Frosst-1.0.3'", "'openff-1.0.0-RC1'", "'openff-1.0.1'", "'openff-1.3.1-alpha.1'", "'openff-1.0.0'", "'openff-1.2.0'", "'openff-2.0.0-rc.2'", "'openff-2.0.0-rc.1'", 
            "'openff-2.0.0'", "'openff-1.1.0'", "'openff-1.3.1'", "'openff-1.1.1'", "'openff-1.0.0-RC2'", "'openff-1.3.0'", "'openff-1.2.1'"]


small_ff_id = small_ff_options.index(Ligand_Force_field)
type_generator = type_generator_options[small_ff_id]
small_ff = small_ff_include[small_ff_id]

# Teach OpenMM about the ligand molecule and force field
ligand_generator = type_generator + "TemplateGenerator(forcefield=" + small_ff + ", molecules=[ligand])"
type_generator_options = eval(ligand_generator)
omm_forcefield.registerTemplateGenerator(type_generator_options.generator)

# Load the receptor structure into Modeller
pdb = openmm.app.PDBFile(receptor)
modeller = openmm.app.Modeller(pdb.topology, pdb.positions)

# Add the ligand
# Under the hood, this line uses the OpenFF Toolkit to generate new parameters for the ligand
modeller.add(ligand_topology.to_openmm(), to_openmm(ligand_positions))

modeller.addHydrogens(omm_forcefield, pH=float(pH))

if Ions == "NaCl":
  positive_ion = 'Na+'
else:
  positive_ion = 'K+'

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model=water_model,
    padding=int(Padding_distance) * unit.angstrom,
    ionicStrength=float(Concentration) * unit.molar, 
    positiveIon=positive_ion, negativeIon='Cl-',
)

# Retrieve the OpenMM Topology, which stores the atoms and connectivity
topology = modeller.getTopology()

# Get the initial positions
# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
positions = modeller.getPositions() + np.array([30, 30, 30]) * unit.angstrom

#Export PDB file
system_pdb = os.path.join(workDir, 'system.pdb')
PDBFile.writeFile(topology, positions, open(system_pdb, 'w'))

# ParmEd's GROMACS exporter can't handle constraints from openmm, so we need a variant for export without them
export_system = omm_forcefield.createSystem(
    modeller.topology,
    # constraints=None,
    # rigidWater=False,
    flexibleConstraints=True
)

# Combine the topology, system and positions into a ParmEd Structure
pmd_complex_struct = pmd.openmm.load_topology(topology, export_system, positions)

# Export AMBER files.
amber_prmtop = os.path.join(workDir, 'system_amber.prmtop')
amber_inpcrd = os.path.join(workDir, 'system_amber.inpcrd')
pmd_complex_struct.save(amber_prmtop, overwrite=True)
pmd_complex_struct.save(amber_inpcrd, overwrite=True)
pprm = pmd.load_file(amber_prmtop, amber_inpcrd)
mprm_from_parmed = mda.Universe(pprm)

# Export GROMACS files.
gromacs_top = os.path.join(workDir, 'system_gromacs.top')
gromacs_gro = os.path.join(workDir, 'system_gromacs.gro')
pmd_complex_struct.save(gromacs_top, overwrite=True)
pmd_complex_struct.save(gromacs_gro, overwrite=True)

pdb_check = os.path.exists(system_pdb)
top_amber = os.path.exists(amber_prmtop)
inpcrd_amber = os.path.exists(amber_inpcrd)
top_gromacs = os.path.exists(gromacs_top)
gro_gromacs = os.path.exists(gromacs_gro)

if pdb_check == True and top_amber == True and inpcrd_amber == True and top_gromacs == True and gro_gromacs == True:
  print("Successfully generated topology! :-)")
else:
  print("ERROR: Check your inputs! ")

## Let's take a look on our simulation box:

In [ ]:
#@title **Show 3D structure**
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

color = "gray" #@param ["gray", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_ligand = True #@param {type:"boolean"}
show_box = True #@param {type:"boolean"}
box_opacity = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}


def show_pdb(show_sidechains=False, show_mainchains=False, show_ligand = False, show_box = False, color="rainbow"):
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(system_pdb,'r').read(),'pdb')

  if color == "gray":
    view.setStyle({'cartoon':{}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  
  if show_box:
    view.addSurface(py3Dmol.SAS, {'opacity': box_opacity, 'color':'white'})
  
  if show_ligand:
    HP = ['UNK']
    view.addStyle({'and':[{'resn':HP}]},
                       {'stick':{'colorscheme':'greenCarbon','radius':0.3}})
    view.setViewStyle({'style':'outline','color':'black','width':0.1})

  view.zoomTo()
  return view


show_pdb(show_sidechains, show_mainchains, show_ligand, show_box, color).show()

In [ ]:
#@title **View and check the Ligand Interaction Network (LigPlot)**
#@markdown This diagram is interactive and allows moving around the residues, as well as clicking the legend to toggle the display of specific residues types or interactions. The diagram will be saved as an HTML file (initial.html).

import MDAnalysis as mda
import prolif as plf
import numpy as np
import os
from prolif.plotting.network import LigNetwork

# load topology
u = mda.Universe(amber_prmtop, system_pdb) 
lig = u.select_atoms("not protein and not (resname HOH) and not (resname CL) and not (resname NA) and not (resname K)")
prot = u.select_atoms("protein")

# create RDKit-like molecules for visualisation
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

fp = plf.Fingerprint()
fp.run(u.trajectory[::10], lig, prot)
df = fp.to_dataframe(return_atoms=True)

net = LigNetwork.from_ifp(df, lmol,
                          # replace with `kind="frame", frame=0` for the other depiction
                          kind="aggregate", frame=0,
                          rotation=270)
net.save(os.path.join(workDir, "initial.html"))
net.display()

In [ ]:
#@title **Open Binding Pose Metadynamics**: evaluating ligand pose stability using metadynamics

#@markdown **OpenBPMD** is an open source implementation of binding pose metadynamics (BPMD). Based on work by [Clark et al, 2016](https://doi.org/10.1021/acs.jctc.6b00201). Primarily built for reranking docked poses out of a list of candidates generated by a docking program. OpenBPMD biases ligands using metadynamics and evaluates how stable they are during the simulation.

# OpenMM
from openmm.app.metadynamics import *

# The rest
import argparse
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import rms, contacts
import mdtraj as md
import pandas as pd
import parmed as pmd
import glob
import os
from sys import stdout, exit, stderr

Google_Drive_Path = '/content/' #@param {type:"string"}
workDir = Google_Drive_Path

#@markdown Name of the output directory:
output = "ligand_pose" #@param {type:"string"}
output = os.path.join(workDir, output)
#@markdown Number of repeat OpenBPMD simulations to run in series:

nreps = "1" #@param ["1","2","3","4","5","6","7","8","9","10"]

#@markdown Size of the metadynamical hill, in kcal/mol:

hill_height = 0.3 #@param {type:"slider", min:0.1, max:1, step:0.1}
#Parameters
lig_resname = 'UNK' # Residue name of the ligand in the structure/parameter file.
nreps = int(nreps) 
parameters = os.path.join(workDir, 'system_amber.prmtop')
structure = os.path.join(workDir, 'system_amber.inpcrd')
continue_simulation = "no" #@param ["yes", "no"]



def main():

    if not os.path.isdir(f'{output}'):
      os.mkdir(f'{output}')
    
    coords = AmberInpcrdFile(structure)
    parm = AmberPrmtopFile(parameters)

    # Minimize
    min_file_name = 'minimized_system.pdb'
    if not os.path.isfile(os.path.join(output,min_file_name)):
        print("Minimizing...")
        #min_pos = minimize(parm, coords.positions, output)
        minimize(parm, coords.positions, output, min_file_name)
    min_pdb = os.path.join(output,min_file_name)

    # Equilibrate
    eq_file_name = 'equil_system.pdb'
    if not os.path.isfile(os.path.join(output,eq_file_name)):
        print("Equilibrating...")
        equilibrate(min_pdb, parm, output, eq_file_name)
    eq_pdb = os.path.join(output,eq_file_name)
    cent_eq_pdb = os.path.join(output,'centred_'+eq_file_name)
    if os.path.isfile(eq_pdb) and not os.path.isfile(cent_eq_pdb):
        mdtraj_top = parameters
        mdu = md.load(eq_pdb, top=mdtraj_top)
        mdu.image_molecules()
        mdu.save_pdb(cent_eq_pdb)

    # Run NREPS number of production simulations
    for idx in range(1, (nreps+1)):
        rep_dir = os.path.join(output,f'rep_{idx}')
        if not os.path.isdir(rep_dir):
            os.mkdir(rep_dir)
        print("Running BPMD simulation " + str(idx))
        if os.path.isfile(os.path.join(rep_dir,'bpm_results.csv')):
            continue
        
        produce(output, idx, lig_resname, eq_pdb, parm, parameters,
                structure, hill_height)
                
        trj_name = os.path.join(rep_dir,'trj.dcd')
                
        PoseScoreArr = get_pose_score(cent_eq_pdb, trj_name, lig_resname)

        ContactScoreArr = get_contact_score(cent_eq_pdb, trj_name, lig_resname)

        # Calculate the CompScore at every frame
        CompScoreArr = np.zeros(99)
        for index in range(ContactScoreArr.shape[0]):
            ContactScore, PoseScore = ContactScoreArr[index], PoseScoreArr[index]
            CompScore = PoseScore - 5 * ContactScore
            CompScoreArr[index] = CompScore

        Scores = np.stack((CompScoreArr, PoseScoreArr, ContactScoreArr), axis=-1)

        # Save a DataFrame to CSV
        df = pd.DataFrame(Scores, columns=['CompScore', 'PoseScore',
                                           'ContactScore'])
        df.to_csv(os.path.join(rep_dir,'bpm_results.csv'), index=False)
                
    collect_results(output, output)

    return None
    

def get_contact_score(structure_file, trajectory_file, lig_resname):
    """A function the gets the ContactScore from an OpenBPMD trajectory.

    Parameters
    ----------
    structure_file : str
        The name of the centred equilibrated system PDB file that 
        was used to start the OpenBPMD simulation.
    trajectory_file : str
        The name of the OpenBPMD trajectory file.
    lig_resname : str
        Residue name of the ligand that was biased.

    Returns
    -------
    contact_scores : np.array 
        ContactScore for every frame of the trajectory.
    """
    u = mda.Universe(structure_file, trajectory_file)

    sel_donor = f"resname {lig_resname} and not name *H*"
    sel_acceptor = f"protein and not name H* and \
                     around 5 resname {lig_resname}"

    # reference groups (first frame of the trajectory, but you could also use
    # a separate PDB, eg crystal structure)
    a_donors = u.select_atoms(sel_donor)
    a_acceptors = u.select_atoms(sel_acceptor)

    cont_analysis = contacts.Contacts(u, select=(sel_donor, sel_acceptor),
                                      refgroup=(a_donors, a_acceptors),
                                      radius=3.5)

    cont_analysis.run()
    # print number of average contacts in the first ns
    # NOTE - hard coded number of frames (100 per traj)
    frame_idx_first_ns = int(len(cont_analysis.timeseries)/10)
    first_ns_mean = np.mean(cont_analysis.timeseries[1:frame_idx_first_ns, 1])
    if first_ns_mean == 0:
        normed_contacts = cont_analysis.timeseries[1:, 1]
    else:
        normed_contacts = cont_analysis.timeseries[1:, 1]/first_ns_mean
    contact_scores = np.where(normed_contacts > 1, 1, normed_contacts)

    return contact_scores


def get_pose_score(structure_file, trajectory_file, lig_resname):
    """A function the gets the PoseScore (ligand RMSD) from an OpenBPMD
    trajectory.

    Parameters
    ----------
    'structure_file : str
        The name of the centred equilibrated system
        PDB file that was used to start the OpenBPMD simulation.
    trajectory_file : str
        The name of the OpenBPMD trajectory file.
    lig_resname : str
        Residue name of the ligand that was biased.

    Returns
    -------
    pose_scores : np.array 
        PoseScore for every frame of the trajectory.
    """
    # Load a MDA universe with the trajectory
    u = mda.Universe(structure_file, trajectory_file)
    # Align each frame using the backbone as reference
    # Calculate the RMSD of ligand heavy atoms
    r = rms.RMSD(u, select='backbone',
                 groupselections=[f'resname {lig_resname} and not name H*'],
                 ref_frame=0).run()
    # Get the PoseScores as np.array
    pose_scores = r.rmsd[1:, -1]

    return pose_scores


def minimize(parm, input_positions, out_dir, min_file_name):
    """An energy minimization function down with an energy tolerance
    of 10 kJ/mol.

    Parameters
    ----------
    parm : Parmed or OpenMM parameter file object
        Used to create the OpenMM System object.
    input_positions : OpenMM Quantity
        3D coordinates of the equilibrated system.
    out_dir : str
        Directory to write the outputs.
    min_file_name : str
        Name of the minimized PDB file to write.
    """
    system = omm_forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometers, constraints=HBonds,)

    # Define platform properties
    platform = Platform.getPlatformByName('CUDA')
    properties = {'CudaPrecision': 'mixed'}

    # Set up the simulation parameters
    # Langevin integrator at 300 K w/ 1 ps^-1 friction coefficient
    # and a 2-fs timestep
    # NOTE - no dynamics performed, but required for setting up
    # the OpenMM system.
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond,
                                    0.002*picoseconds)
    simulation = Simulation(topology, system, integrator, platform,
                            properties)
    simulation.context.setPositions(input_positions)

    # Minimize the system - no predefined number of steps
    simulation.minimizeEnergy()

    # Write out the minimized system to use w/ MDAnalysis
    positions = simulation.context.getState(getPositions=True).getPositions()
    out_file = os.path.join(out_dir,min_file_name)
    PDBFile.writeFile(simulation.topology, positions,
                      open(out_file, 'w'))

    return None


def equilibrate(min_pdb, parm, out_dir, eq_file_name):
    """A function that does a 500 ps NVT equilibration with position
    restraints, with a 5 kcal/mol/A**2 harmonic constant on solute heavy
    atoms, using a 2 fs timestep.

    Parameters
    ----------
    min_pdb : str
        Name of the minimized PDB file.
    parm : Parmed or OpenMM parameter file object
        Used to create the OpenMM System object.
    out_dir : str
        Directory to write the outputs to.
    eq_file_name : str
        Name of the equilibrated PDB file to write.
    """
    # Get the solute heavy atom indices to use
    # for defining position restraints during equilibration
    universe = mda.Universe(min_pdb,
                            format='XPDB', in_memory=True)
    solute_heavy_atom_idx = universe.select_atoms('not resname WAT and\
                                                   not resname SOL and\
                                                   not resname HOH and\
                                                   not resname CL and \
                                                   not resname NA and \
                                                   not resname K and \
                                                   not name H*').indices
    # Necessary conversion to int from numpy.int64,
    # b/c it breaks OpenMM C++ function
    solute_heavy_atom_idx = [int(idx) for idx in solute_heavy_atom_idx]

    # Add the restraints.
    # We add a dummy atoms with no mass, which are therefore unaffected by
    # any kind of scaling done by barostat (if used). And the atoms are
    # harmonically restrained to the dummy atom. We have to redefine the
    # system, b/c we're adding new particles and this would clash with
    # modeller.topology.
    system = omm_forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometers, constraints=HBonds)

    # Add the harmonic restraints on the positions
    # of specified atoms
    restraint = HarmonicBondForce()
    restraint.setUsesPeriodicBoundaryConditions(True)
    system.addForce(restraint)
    nonbonded = [force for force in system.getForces()
                 if isinstance(force, NonbondedForce)][0]
    dummyIndex = []
    input_positions = PDBFile(min_pdb).getPositions()
    positions = input_positions
    # Go through the indices of all atoms that will be restrained
    for i in solute_heavy_atom_idx:
        j = system.addParticle(0)
        # ... and add a dummy/ghost atom next to it
        nonbonded.addParticle(0, 1, 0)
        # ... that won't interact with the restrained atom 
        nonbonded.addException(i, j, 0, 1, 0)
        # ... but will be have a harmonic restraint ('bond')
        # between the two atoms
        restraint.addBond(i, j, 0*nanometers,
                          5*kilocalories_per_mole/angstrom**2)
        dummyIndex.append(j)
        input_positions.append(positions[i])

    integrator = LangevinIntegrator(300*kelvin, 1/picosecond,
                                    0.002*picoseconds)
    platform = Platform.getPlatformByName('CUDA')
    properties = {'CudaPrecision': 'mixed'}
    sim = Simulation(topology, system, integrator,
                     platform, properties)
    sim.context.setPositions(input_positions)
    integrator.step(250000)  # run 500 ps of equilibration
    all_positions = sim.context.getState(
        getPositions=True, enforcePeriodicBox=True).getPositions()
    # we don't want to write the dummy atoms, so we only
    # write the positions of atoms up to the first dummy atom index
    relevant_positions = all_positions[:dummyIndex[0]]
    out_file = os.path.join(out_dir,eq_file_name)
    PDBFile.writeFile(sim.topology, relevant_positions,
                      open(out_file, 'w'))

    return None


def produce(out_dir, idx, lig_resname, eq_pdb, parm, parm_file,
            coords_file, set_hill_height):
    """An OpenBPMD production simulation function. Ligand RMSD is biased with
    metadynamics. The integrator uses a 4 fs time step and
    runs for 10 ns, writing a frame every 100 ps.

    Writes a 'trj.dcd', 'COLVAR.npy', 'bias_*.npy' and 'sim_log.csv' files
    during the metadynamics simulation in the '{out_dir}/rep_{idx}' directory.
    After the simulation is done, it analyses the trajectories and writes a
    'bpm_results.csv' file with time-resolved PoseScore and ContactScore.

    Parameters
    ----------
    out_dir : str
        Directory where your equilibration PDBs and 'rep_*' dirs are at.
    idx : int
        Current replica index.
    lig_resname : str
        Residue name of the ligand.
    eq_pdb : str
        Name of the PDB for equilibrated system.
    parm : Parmed or OpenMM parameter file object
        Used to create the OpenMM System object.
    parm_file : str
        The name of the parameter or topology file of the system.
    coords_file : str
        The name of the coordinate file of the system.
    set_hill_height : float
        Metadynamic hill height, in kcal/mol.

    """
    # First, assign the replica directory to which we'll write the files
    write_dir = os.path.join(out_dir,f'rep_{idx}')
    # Get the anchor atoms by ...
    universe = mda.Universe(eq_pdb,
                            format='XPDB', in_memory=True)
    # ... finding the protein's COM ...
    prot_com = universe.select_atoms('protein').center_of_mass()
    x, y, z = prot_com[0], prot_com[1], prot_com[2]
    # ... and taking the heavy backbone atoms within 5A of the COM
    sel_str = f'point {x} {y} {z} 5 and backbone and not name H*'
    anchor_atoms = universe.select_atoms(sel_str)
    # ... or 10 angstrom
    if len(anchor_atoms) == 0:
        sel_str = f'point {x} {y} {z} 10 and backbone and not name H*'
        anchor_atoms = universe.select_atoms(sel_str)

    anchor_atom_idx = anchor_atoms.indices.tolist()

    # Get indices of ligand heavy atoms
    lig = universe.select_atoms(f'resname {lig_resname} and not name H*')

    lig_ha_idx = lig.indices.tolist()

    # Set up the system to run metadynamics
    if continue_simulation == "yes":
      topology = PDBFile(eq_pdb).getTopology()
      with open(os.path.join(workDir, 'system_openMM.xml')) as input:
        system = XmlSerializer.deserialize(input.read())
    else:
      topology = modeller.getTopology()
      system = omm_forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometers, constraints=HBonds, hydrogenMass=4*amu)
      openMM_system = os.path.join(workDir, 'system_openMM.xml')
      openMM_system_check = os.path.exists(openMM_system)
      if openMM_system_check == False:
        with open(os.path.join(workDir, 'system_openMM.xml'), 'w') as output:
          output.write(XmlSerializer.serialize(system))
      else:
        pass
  
    # get the atom positions for the system from the equilibrated
    # system
    input_positions = PDBFile(eq_pdb).getPositions()

    # Add an 'empty' flat-bottom restraint to fix the issue with PBC.
    # Without one, RMSDForce object fails to account for PBC.
    k = 0*kilojoules_per_mole  # NOTE - 0 kJ/mol constant
    upper_wall = 10.00*nanometer
    fb_eq = '(k/2)*max(distance(g1,g2) - upper_wall, 0)^2'
    upper_wall_rest = CustomCentroidBondForce(2, fb_eq)
    upper_wall_rest.addGroup(lig_ha_idx)
    upper_wall_rest.addGroup(anchor_atom_idx)
    upper_wall_rest.addBond([0, 1])
    upper_wall_rest.addGlobalParameter('k', k)
    upper_wall_rest.addGlobalParameter('upper_wall', upper_wall)
    upper_wall_rest.setUsesPeriodicBoundaryConditions(True)
    system.addForce(upper_wall_rest)

    alignment_indices = lig_ha_idx + anchor_atom_idx

    rmsd = RMSDForce(input_positions, alignment_indices)
    # Set up the typical metadynamics parameters
    grid_min, grid_max = 0.0, 1.0  # nm
    hill_height = set_hill_height*kilocalories_per_mole
    hill_width = 0.002  # nm, also known as sigma

    grid_width = hill_width / 5
    # 'grid' here refers to the number of grid points
    grid = int(abs(grid_min - grid_max) / grid_width)

    rmsd_cv = BiasVariable(rmsd, grid_min, grid_max, hill_width,
                           False, gridWidth=grid)

    # define the metadynamics object
    # deposit bias every 1 ps, BF = 4, write bias every ns
    meta = Metadynamics(system, [rmsd_cv], 300.0*kelvin, 4.0, hill_height,
                        250, biasDir=write_dir,
                        saveFrequency=250000)

    # Set up and run metadynamics
    integrator = LangevinIntegrator(300*kelvin, 1.0/picosecond,
                                    0.004*picoseconds)
    platform = Platform.getPlatformByName('CUDA')
    properties = {'CudaPrecision': 'mixed'}

    simulation = Simulation(topology, system, integrator, platform,
                            properties)
    simulation.context.setPositions(input_positions)

    trj_name = os.path.join(write_dir,'trj.dcd')

    sim_time = 10  # ns
    steps = 250000 * sim_time

    simulation.reporters.append(DCDReporter(trj_name, 25000))  # every 100 ps
    simulation.reporters.append(StateDataReporter(stdout, 25000,
                                step=True, speed=True, progress=True,
                                totalSteps=steps, remainingTime=True, 
                                separator='\t\t'))
    simulation.reporters.append(StateDataReporter(
                                os.path.join(write_dir,'sim_log.csv'), 250000,
                                step=True, temperature=True, progress=True,
                                remainingTime=True, speed=True,
                                totalSteps=steps, separator=','))  # every 1 ns
    
    print("\n> Simulating " + str(steps) + " steps...")

    colvar_array = np.array([meta.getCollectiveVariables(simulation)])
    for i in range(0, int(steps), 500):
        if i % 25000 == 0:
            # log the stored COLVAR every 100ps
            np.save(os.path.join(write_dir,'COLVAR.npy'), colvar_array)
        meta.step(simulation, 500)
        current_cvs = meta.getCollectiveVariables(simulation)
        # record the CVs every 2 ps
        colvar_array = np.append(colvar_array, [current_cvs], axis=0)
    np.save(os.path.join(write_dir,'COLVAR.npy'), colvar_array)
    # center everything using MDTraj, to fix any PBC imaging issues
    # mdtraj can't use GMX TOP, so we have to specify the GRO file instead
    mdtraj_top = parm_file
    mdu = md.load(trj_name, top=mdtraj_top)
    mdu.image_molecules()
    mdu.save(trj_name)

    return None


def collect_results(in_dir, out_dir):
    """A function that collects the time-resolved BPM results,
    takes the scores from last 2 ns of the simulation, averages them
    and writes that average as the final score for a given pose.

    Writes a 'results.csv' file in 'out_dir' directory.
    
    Parameters
    ----------
    in_dir : str
        Directory with 'rep_*' directories.
    out_dir : str
        Directory where the 'results.csv' file will be written
    """
    compList = []
    contactList = []
    poseList = []
    # find how many repeats have been run
    glob_str = os.path.join(in_dir,'rep_*')
    nreps = len(glob.glob(glob_str))
    for idx in range(1, (nreps+1)):
        f = os.path.join(in_dir,f'rep_{idx}','bpm_results.csv')
        df = pd.read_csv(f)
        # Since we only want last 2 ns, get the index of
        # the last 20% of the data points
        last_2ns_idx = round(len(df['CompScore'].values)/5)  # round up
        compList.append(df['CompScore'].values[-last_2ns_idx:])
        contactList.append(df['ContactScore'].values[-last_2ns_idx:])
        poseList.append(df['PoseScore'].values[-last_2ns_idx:])

    # Get the means of the last 2 ns
    meanCompScore = np.mean(compList)
    meanPoseScore = np.mean(poseList)
    meanContact = np.mean(contactList)
    # Get the standard deviation of the final 2 ns
    meanCompScore_std = np.std(compList)
    meanPoseScore_std = np.std(poseList)
    meanContact_std = np.std(contactList)
    # Format it the Pandas way
    d = {'CompScore': [meanCompScore], 'CompScoreSD': [meanCompScore_std],
         'PoseScore': [meanPoseScore], 'PoseScoreSD': [meanPoseScore_std],
         'ContactScore': [meanContact], 'ContactScoreSD': [meanContact_std]}

    results_df = pd.DataFrame(data=d)
    results_df = results_df.round(3)
    results_df.to_csv(os.path.join(out_dir,'results.csv'), index=False)

main()

##  **Analysing BPMD simulations**

Understanding the BPMD results:

- The lower (more negative) the CompScore, the more likely a given pose to be the correct pose. 'Correct' here means a binding pose that is similar to a pose observed in an experimentally determined structure.

- PoseScore, being a more objective measure than ContactScore, should be given a slighlty higher weight when the CompScores of two poses are very similar.

- The standard deviations of the scores might seem really high when compared to the scores themselves. Short metadynamics simulations are known to be very noisy and this is expected. Poses with lower standard deviation also tend to be have a lower RMSD to the known pose. Therefore, standard deviations is a useful indicator of confidence.

- CompScores should only be compared between poses of the same ligand, not for comparing stability of different ligands.

In [ ]:
#@title  **Plot the time-wise BPMD scores for each replica:**
#@markdown Select the replica:

replica = "1" #@param ["1","2","3","4","5","6","7","8","9","10"]

#@markdown **Important:** Probably, the scores will be very noisy. This is typical of short metadynamics simulations that haven't yet converged the free energy surface. However, we're not interested in the free energy. We're only want to test the stability of the ligand in the binding pose.
if int(replica) > int(nreps):
  print("You have less replicas than requested.")
  exit()
else:
  pass

f = os.path.join(workDir,output,'rep_' + str(int(replica)),'bpm_results.csv')
df = pd.read_csv(f)
time_sequence = np.linspace(0,10,99)

plt.title('CompScore from ' + str(replica))
plt.plot(time_sequence,df['CompScore'])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('CompScore')
plt.ylim(-5,5)
plt.show()
plt.savefig(os.path.join(workDir, "CompScore_" + str(replica) + "_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(df['CompScore'])
raw_data.to_csv(os.path.join(workDir, "CompScore_" + str(replica) + "_BPMD.csv"))

plt.title('PoseScore from 1 ' + str(replica))
plt.plot(time_sequence,df['PoseScore'],color='darkorange')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('PoseScore')
plt.ylim(0,5)
plt.show()
plt.savefig(os.path.join(workDir, "PoseScore_" + str(replica) + "_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(df['PoseScore'])
raw_data.to_csv(os.path.join(workDir, "PoseScore_" + str(replica) + "_BPMD.csv"))

plt.title('ContactScore from 1 ' + str(replica))
plt.plot(time_sequence,df['ContactScore'],color='green')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('ContactScore')
plt.ylim(-0.1,1.1)
plt.show()
plt.savefig(os.path.join(workDir, "ContactScore_" + str(replica) + "_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(df['ContactScore'])
raw_data.to_csv(os.path.join(workDir, "ContactScore_" + str(replica) + "_BPMD.csv"))

In [ ]:
#@title  **Plot the time-wise BPMD scores average over multiple replicas :**
#@markdown **Important:** To have more confidence in the stability scores, we run multiple repeat simulations. In the final PoseScore and ContactScore, after N repeats, we take the mean of the score over the last 2 ns, which also helps with the noise.

# We'll store the results from 10 repeats in a 10x99 matrix.
CompScores = np.zeros((10,99))
PoseScores = np.zeros((10,99))
ContactScores = np.zeros((10,99))

# Fill those matrices with the scores from each repeat
for idx in range(1,(nreps+1)):
    f = os.path.join(workDir,output,f'rep_{idx}','bpm_results.csv')
    df = pd.read_csv(f)
    CompScores[idx] = df['CompScore']
    PoseScores[idx] = df['PoseScore']
    ContactScores[idx] = df['ContactScore']

# Average out the scores from all of the repeats,
# giving a mean of the scores at each frame of the trajectory
averagedCompScore = np.array([ np.mean(CompScores[:,i]) for i in range(0,99) ])
averagedPoseScore = [ np.mean(PoseScores[:,i]) for i in range(0,99) ]
averagedContactScore = [ np.mean(ContactScores[:,i]) for i in range(0,99) ]
# Get the standard deviation for the CompScore
CompScore_stddev = np.array([ np.std(CompScores[:,i]) for i in range(0,99) ])
# An array of time steps for plotting the x axis
time_sequence = np.linspace(0,10,99)

plt.title('CompScore from ' + str(nreps) + ' repeats')
plt.plot(time_sequence,averagedCompScore, color='blue')
# Visualise the standard deviation of CompScore at each frame
plt.fill_between(time_sequence, averagedCompScore-CompScore_stddev, averagedCompScore+CompScore_stddev, 
                 color='blue', alpha=0.3, lw=0)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('CompScore')
plt.ylim(-5,5)
plt.show()
plt.savefig(os.path.join(workDir, "CompScore_mean_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(averagedCompScore)
raw_data.to_csv(os.path.join(workDir, "CompScore_mean_BPMD.csv"))

plt.title('PoseScore from ' + str(nreps) + ' repeats')
time_sequence = np.linspace(0,10,99)
plt.plot(time_sequence,averagedPoseScore,color='darkorange')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('PoseScore')
plt.ylim(0,5)
plt.show()
plt.savefig(os.path.join(workDir, "PoseScore_mean_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(averagedPoseScore)
raw_data.to_csv(os.path.join(workDir, "PoseScore_mean_BPMD.csv"))

plt.title('ContactScore from ' + str(nreps) + ' repeats')
time_sequence = np.linspace(0,10,99)
plt.plot(time_sequence,averagedContactScore,color='green')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('time(ns)')
plt.ylabel('ContactScore')
plt.ylim(-0.1,1.1)
plt.show()
plt.savefig(os.path.join(workDir, "ContactScore_mean_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(averagedContactScore)
raw_data.to_csv(os.path.join(workDir, "ContactScore_mean_BPMD.csv"))

compList = []
contactList = []
poseList = []
# Find how many repeats have been run
glob_str = os.path.join(workDir,output,'rep_*')
nreps = len(glob.glob(glob_str))
for idx in range(1, (nreps+1)):
    f = os.path.join(workDir,output,f'rep_{idx}','bpm_results.csv')
    df = pd.read_csv(f)
    # Since we only want last 2 ns, get the index of
    # the last 20% of the data points
    last_2ns_idx = round(len(df['CompScore'].values)/5)  # round up
    compList.append(df['CompScore'].values[-last_2ns_idx:])
    contactList.append(df['ContactScore'].values[-last_2ns_idx:])
    poseList.append(df['PoseScore'].values[-last_2ns_idx:])

# Get the means of the last 2 ns
meanCompScore = np.mean(compList)
meanPoseScore = np.mean(poseList)
meanContact = np.mean(contactList)
# Get the standard deviation of the final 2 ns
meanCompScore_std = np.std(compList)
meanPoseScore_std = np.std(poseList)
meanContact_std = np.std(contactList)
# Format it the Pandas way
d = {'CompScore': [meanCompScore], 'CompScoreSD': [meanCompScore_std],
     'PoseScore': [meanPoseScore], 'PoseScoreSD': [meanPoseScore_std],
     'ContactScore': [meanContact], 'ContactScoreSD': [meanContact_std]}

results_df = pd.DataFrame(data=d)
results_df = results_df.round(3)
print("In order to get a single number that evaluates a given pose/ligand, we take the scores of the final 2 ns.")
results_df

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
import subprocess
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFMCS,AllChem, Draw, PandasTools
import seaborn as sns
#@title **Evaluate the poses over the ensemble with the pre-trained GNINA models**
#@markdown Pose (CNNscore) and binding affinity (Affinity Score and CNNaffinity) predictions with GNINA models.

# @markdown Minimize ligands already positioned in a binding site:
minimization = "Yes" #@param ["Yes","No"]
Skip = 30 #@param {type:"slider", min:1, max:100, step:1}
pdb_dir = os.path.join(workDir,'PDBs/')
pdb_dir_check = os.path.exists(pdb_dir)
if pdb_dir_check == False:
  os.mkdir(pdb_dir)
else:
  pass

flist = []
pdb = os.path.join(output,f"centred_equil_system.pdb")
ref = os.path.join(str(output) + '/rep_' + str(1) + '/trj.dcd')
for i in range(1, int(nreps+1)):
  flist.append(os.path.join(str(output) + '/rep_' + str(i) + '/trj.dcd'))

u1 = mda.Universe(pdb, flist)
u2 = mda.Universe(pdb, ref)
u2.trajectory[0] # set u2 to first frame

align.AlignTraj(u1, u2, select='name CA', in_memory=True).run()

#Save a trajectory with protein + ligand in PDB format
protein_ligand = u1.select_atoms("not (resname HOH) and not (resname NA) and not (resname CL) and not (resname K)")
with mda.Writer(os.path.join(output,f'protein_ligand_GNINA.dcd'), protein_ligand.n_atoms) as W:
  for ts in u1.trajectory[0:len(u1.trajectory):int(Skip)]:
    W.write(protein_ligand)

if minimization == 'Yes':
  minimize = "--minimize"
else:
  minimize = "--score_only"

affinity = []
CNNscore = []
CNNaffinity = []
protein_atoms = u1.select_atoms("protein")
ligand_atoms = u1.select_atoms("resname UNK")
i = 0
for ts in u1.trajectory[0:len(u1.trajectory):int(Skip)]: 
  if i > -1:
    with mda.Writer(os.path.join(pdb_dir,'protein.pdb'), protein_atoms.n_atoms) as W:
      W.write(protein_atoms)
    with mda.Writer(os.path.join(pdb_dir,'ligand.pdb'), ligand_atoms.n_atoms) as W:
      W.write(ligand_atoms)    
  gnina_score = "./gnina -r " + str(pdb_dir) + "protein.pdb" + " -l " +  str(pdb_dir) + "ligand.pdb " + str(minimize) + " -o score_gnina.sdf"
  original_stdout = sys.stdout # Save a reference to the original standard output
  with open('gnina_score.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(gnina_score)
    sys.stdout = original_stdout # Reset the standard output to its original value
  print("Running frame "+ str(i))
  subprocess.run(["chmod 700 gnina_score.sh"], shell=True)
  subprocess.run(["./gnina_score.sh"], shell=True)
  result=Chem.SDMolSupplier("score_gnina.sdf",True)
  affinity.append(float(result[0].GetProp('minimizedAffinity')))
  CNNscore.append(float(result[0].GetProp('CNNscore')))
  CNNaffinity.append(float(result[0].GetProp('CNNaffinity')))
  i = i + 1

raw_data=pd.DataFrame(affinity)
raw_data.to_csv(os.path.join(output,f"GNINA_affinity.csv"))
raw_data=pd.DataFrame(CNNscore)
raw_data.to_csv(os.path.join(output,f"GNINA_CNNscore.csv"))
raw_data=pd.DataFrame(CNNaffinity)
raw_data.to_csv(os.path.join(output,f"GNINA_CNNaffinity.csv"))

In [ ]:
#@title **Plot the Affinity Score, CNNscore and CNNaffinity, calculated with GNINA models**


time_sequence = np.linspace(0,int(len(u1.trajectory)/int(Skip)),int(len(u1.trajectory)/int(Skip)))

plt.title('Affinity Score')
plt.plot(time_sequence,affinity)
plt.xlim(0, time_sequence[-1])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('Frames')
plt.ylabel('Affinity (kcal/mol)')
plt.show()
plt.savefig(os.path.join(workDir, "Affinity_Score_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(affinity)
raw_data.to_csv(os.path.join(workDir, "Affinity_Score_BPMD.csv"))

plt.title('CNNscore')
plt.plot(time_sequence,CNNscore,color='darkorange')
plt.xlim(0, time_sequence[-1])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('Frames')
plt.ylabel('CNNscore')
plt.show()
plt.savefig(os.path.join(workDir, "CNNscore_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(CNNscore)
raw_data.to_csv(os.path.join(workDir, "CNNscore_BPMD.csv"))

plt.title('CNNaffinity')
plt.plot(time_sequence,CNNaffinity,color='green')
plt.xlim(0, time_sequence[-1])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('Frames')
plt.ylabel('CNNaffinity')
plt.show()
plt.savefig(os.path.join(workDir, "CNNaffinity_BPMD.png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(CNNaffinity)
raw_data.to_csv(os.path.join(workDir, "CNNaffinity_BPMD.csv"))

In [ ]:
#@title **Best pose selection from BPMD simulations:**

Frame = "98" #@param {type:"string"}

if int(Frame) > (len(u1.trajectory)/int(Skip)-int(1)):
  print("You didn't save enough frames in your trajectory. Please, run the previous cell again with a lower skip.")
  exit()
else:
  pass

protein_ligand = u1.select_atoms("not (resname HOH) and not (resname NA) and not (resname CL) and not (resname K)")
with mda.Writer(os.path.join(output,f'protein_ligand_' + str(Frame) + '.pdb'), protein_ligand.n_atoms) as W:
  for ts in u1.trajectory[int(Frame):(int(Frame)+1):1]:
    W.write(protein_ligand)


view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.05})

view.addModel(open(os.path.join(output,f'protein_ligand_' + str(Frame) + '.pdb'),'r').read(),'pdb')
Prot=view.getModel()
# Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'},'stick':{'radius':.1}})
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.2,'color':'white'})

print ('Affinity: {}'.format(affinity[(int(Frame)-1)]))
print ('CNN Score: {}'.format(CNNscore[(int(Frame)-1)]))
print ('CNN Affinity: {}'.format(CNNaffinity[(int(Frame)-1)]))

HP = ['UNK']
view.addStyle({'and':[{'resn':HP}]},
              {'stick':{'colorscheme':'greenCarbon','radius':0.3}})
view.setViewStyle({'style':'outline','color':'black','width':0.1})


view.zoomTo()
view.show()

In [ ]:
#@title **View and check the Ligand Interaction Network (LigPlot) on the best pose**
#@markdown This diagram is interactive and allows moving around the residues, as well as clicking the legend to toggle the display of specific residues types or interactions. The diagram will be saved as an HTML file (frame_number.html).
Frame = "19" #@param {type:"string"}

if int(Frame) > (len(u1.trajectory)/int(Skip)-int(1)):
  print("You didn't save enough frames in your trajectory. Please, run the previous cell again with a lower skip.")
  exit()
else:
  pass

import MDAnalysis as mda
import prolif as plf
import numpy as np
import os
from prolif.plotting.network import LigNetwork


protein_ligand = u1.select_atoms("all")
with mda.Writer(os.path.join(output,f'protein_ligand_' + str(Frame) + '.pdb'), protein_ligand.n_atoms) as W:
  for ts in u1.trajectory[int(Frame):(int(Frame)+1):1]:
    W.write(protein_ligand)

# load topology
u = mda.Universe(os.path.join(workDir,"system_amber.prmtop"), os.path.join(output,f'protein_ligand_' + str(Frame) + '.pdb')) 
lig = u.select_atoms("not protein and not (resname HOH) and not (resname CL) and not (resname NA) and not (resname K)")
prot = u.select_atoms("protein")

# create RDKit-like molecules for visualisation
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

fp = plf.Fingerprint()
fp.run(u.trajectory[::5], lig, prot)
df = fp.to_dataframe(return_atoms=True)

net = LigNetwork.from_ifp(df, lmol,
                          # replace with `kind="frame", frame=0` for the other depiction
                          kind="aggregate", frame=0,
                          rotation=270)
net.save(os.path.join(workDir, "frame_" + str(Frame) + ".html"))
net.display()

In [ ]:
#@title **Compute RMSD of the ligand during BPMD simulations**
#@markdown **Provide output file names below:** 
import MDAnalysis.analysis.rms
Output_name = 'rmsd_lig' #@param {type:"string"}


u = mda.Universe(os.path.join(output,f"centred_equil_system.pdb"), flist)
ref = mda.Universe(os.path.join(output,f"centred_equil_system.pdb"), os.path.join(str(output) + '/rep_' + str(1) + '/trj.dcd'))
ref.trajectory[0] # set u2 to first frame


#Save a trajectory with protein + ligand in PDB format
R = MDAnalysis.analysis.rms.RMSD(u, ref,
           select="resname UNK and not (name H*)",
           groupselections=["resname UNK and not (name H*)"])                       
R.run()

# Plotting:
rmsd = R.rmsd.T   # transpose makes it easier for plotting
time = rmsd[0]/10

ax = plt.plot(time, rmsd[2], alpha=0.6, color = 'blue', linewidth = 1.0)
# plt.xlim(0, time[-1])
# plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsd[2])
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot RMSD as a ditribution**

#@markdown **Provide output file names below:** 
Output_name = 'rmsd_dist' #@param {type:"string"}

ax = sb.kdeplot(rmsd[2], color="blue", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')